In [5]:
# Load dataset
import netCDF4 as nc

# Open the file
ds = nc.Dataset('./power_901_annual_radiation_utc.nc.nc4')

In [6]:
# Print the variables and dimensions in the file
print(ds.variables)

{'time': <class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: years since 1984-12-31 00:00:00
    long_name: time
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of -2147483647 used, 'lat': <class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    _FillValue: -999.0
    long_name: latitude
    standard_name: latitude
    units: degrees_north
unlimited dimensions: 
current shape = (180,)
filling on, 'lon': <class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    _FillValue: -999.0
    long_name: longitude
    standard_name: longitude
    units: degrees_east
unlimited dimensions: 
current shape = (360,)
filling on, 'ALLSKY_SFC_SW_DWN': <class 'netCDF4._netCDF4.Variable'>
float64 ALLSKY_SFC_SW_DWN(time, lat, lon)
    _FillValue: -999.0
    long_name: All Sky Surface Shortwave Downward Irradiance
    standard_name: All_Sky_Surface_Shortwave_Downward_Irradiance
    units: W m-2
    valid_max: 1500.0
    valid_min: 0.0
    valid_range: [   0. 1500.]
unlim

In [7]:
print(ds.dimensions)

{'time': <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 2, 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 180, 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 360}


In [8]:
# print lowest and highest longitude and latitude
print(ds.variables['lon'][:].min(), ds.variables['lon'][:].max())
# print lowest and highest time
print(ds.variables['time'][:].min(), ds.variables['time'][:].max())

-179.5 179.5
35 36


In [9]:
from numpy.ma import masked

finalOutput = dict()

timeItems = ds.variables['time'][:]
solarItems = ds.variables['ALLSKY_SFC_SW_DWN'][:]
latItems = ds.variables['lat'][:]
lonItems = ds.variables['lon'][:]

for x in range(len(timeItems)):
    for y in range(len(latItems)):
        for z in range(len(lonItems)):
            val = solarItems[x][y][z]
            if val is masked:
                continue

            lat = float(latItems[y])
            lon = float(lonItems[z])
            if lat not in finalOutput:
                finalOutput[lat] = dict()
            if lon not in finalOutput[lat]:
                finalOutput[lat][lon] = [0, 0]
            finalOutput[lat][lon][0] += val
            finalOutput[lat][lon][1] += 1
results = []
for lat in finalOutput:
    for lon in finalOutput[lat]:
        results.append([lat, lon, finalOutput[lat][lon][0] / finalOutput[lat][lon][1]])
results.sort()
with open("radiation.csv", "w") as file:
    file.write(",".join(["lat", "lon", "ALLSKY_SFC_SW_DWN"]))
    file.write("\n")
    for result in results:
        file.write(",".join([str(x) for x in result]))
        file.write("\n")            
